### RAA_시장지표

    1. 매체 : 
        a.einfomax
        b.Naver Finance
        c.Naver 거시경제지표

#### einfomax
    1.Web : https://economic.einfomax.co.kr/sh.html#/
    
    2.Selenium + Beautifulsoup 사용
    
    3.현 시점을 기준으로 한달씩 검색
    
    4.중요도(impact)의 경우에는 해당 지표의 중요도에만 "display: none;"이 아님
    
    5.국가명 > 국가코드
        a.ZAA009M00의 NATN_CODE로 통일
        b.유로존의 경우 국제기구로 등록

In [9]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd

import datetime
import time
import requests
import os
from io import BytesIO

from dateutil.relativedelta import relativedelta 

In [18]:
delta = relativedelta(months=1)
now = datetime.date.fromisoformat("2021-07-31")
date,cur,nation,title,index,tot,impact = [],[],[],[],[],[],[]

In [23]:
driver = webdriver.Chrome('C:/Users/shic/chromedriver_win32/chromedriver.exe')
driver.get(url='https://economic.einfomax.co.kr/sh.html#/')
driver.find_element_by_xpath("/html/body/div/div/div[1]/div[1]/div[3]/div[1]/div[1]/div/select/option[3]").click()
driver.implicitly_wait(10)
driver.find_element_by_xpath("/html/body/div/div/div[1]/div[1]/div[2]/a[2]").click()
driver.find_element_by_xpath("/html/body/div/div/div[2]/div/div[1]/div[1]/div[1]/div[2]/button[3]").click()
driver.find_element_by_xpath("/html/body/div/div/div[2]/div/div[1]/div[3]/button[1]").click()
driver.implicitly_wait(10)
for _ in range(180):
    soup = BeautifulSoup(driver.page_source,"html.parser")
    soup = soup.find_all("table",class_= "eco-table fontsize12")[1].find_all("tr",class_ = "im-tr ng-scope")
    for s in soup:
        tmp = [x.text for x in s.find_all("td")]
        date.append(str(now.year)+'/'+tmp[0])
        cur.append(tmp[2])
        nation.append(tmp[4])
        title.append(tmp[6])
        index.append(tmp[8])
        impact.append([x.get("ng-show") for x in s.find_all("td",class_="text-center")[6].find_all("span") if x.get("style") != "display: none;"][0])
        tot.append(s)
    driver.find_element_by_xpath("/html/body/div/div/div[1]/div[1]/div[3]/div[1]/div[2]/div[1]").click()
    driver.implicitly_wait(10)
    now -= delta

In [249]:
def ftn(x):
    if x != x:
        return None#,x
    if x == "":
        return None
    try :
        return np.float64(x)
    except:
        for num in range(len(x)):
            if x[num].isdigit() or x[num] == "-" or x[num] == "." or x[num] == ",":
                continue
            else:
                break
        return np.float64(x[:num].replace(",",""))#,x[num:]

In [261]:
einfomax = pd.DataFrame({
    "Date" : date,
    "Cur" : cur,
    "Nation" : nation,
    "Title" : title,
    "index" : index,
#     "tot" : tot,
    "impact" : impact
})

einfomax.Date = einfomax.Date.map(lambda x: x.replace("/",""))
einfomax.impact = einfomax.impact.map(lambda x:x.split("'")[1])
einfomax.Title = einfomax.Title.map(lambda x:x.strip())
dic = {'1':'3',
      '2':'2',
      '3':'1'}
einfomax.impact = einfomax.impact.map(lambda x:dic[x])

einfomax = einfomax.rename(columns={
    "Date" : "기준일자",
    "Nation" : "국가명",
    "Title" : "지표명",
    "index" : "지표내용",
    "impact" : "중요도구분코드"
})
einfomax['지표값'] = einfomax.지표내용.map(lambda x:ftn(x))

In [262]:
nation_dic = {
    '그리스' : '300',
    '남아공' : '710',
    '네덜란드' : '528',
    '뉴질랜드' : '554',
    '대만' : '158',
    '말레이시아' : '458',
    '스위스' : '756',
    '스페인' : '724',
    '이탈리아' : '380',
    '인도네시아' : '360',
    '체코' : '203',
    '터키' : '792',
    '핀란드' : '246',
    '헝가리' : '348',
    '홍콩' : '344',
    '러시아' : '643',
    '브라질' : '76',
    '인도' : '356',
    '한국' : '410',
    '독일' : '276',
    '영국' : '826',
    '유로존' : '998',
    '일본' : '392',
    '중국' : '156',
    '호주' : '36',
    '오스트리아' : '40',
    '미국' : '840',
    '벨기에' : '56',
    '캐나다' : '124',
    '프랑스' : '250',
    '페루' : '604',
    '폴란드' : '616',
    '가나' : '288',
    '태국' : '764',
    '덴마크' : '208',
    '스웨덴' : '752',
    '싱가포르' : '702',
    '카타르' : '634',
    '라트비아' : '428',
    '잠비아' : '894',
    '필리핀' : '608',
    '칠레' : '152',
    '아르헨티나' : '32',
    '노르웨이' : '578',
    '포르투칼' : '620',
    '에티오피아' : '231',
    '보츠와나' : '72',
    '콜롬비아' : '170',
    '아일랜드' : '372',
    '에스토니아' : '233',
    '루마니아' : '642',
    '우크라이나' : '804',
    '쿠웨이트' : '414',
    '앙골라' : '24',
    '멕시코' : '484',
    '케냐' : '404',
    '남아프리카' : '710',
    '우간다' : '800',
    '이스라엘' : '376',
    '슬로바키아': '703',
    '리투아니아' : '440',
    '탄자니아' : '834',
    '이집트' : '818',
    '사우디아라비아' : '682',
    '르완다' : '646',
    '요르단' : '400',
    '슬로베니아' : '705',
    '아이슬란드' : '352',
    '짐바브웨' : '716',
    '바레인' : '48',
    '나이지리아' : '566',
    '모잠비크' : '508',
    '룩셈부르크' : '442',
    '크로아티아' : '191',
    '나미비아' : '516',
    '카자흐스탄' : '398',
    '말라위' : '454',
    '오만' : '512',
    '모리셔스' : '480',
    '세르비아' : '688',
    '아랍' : '784'
}

In [263]:
einfomax['국가코드'] = einfomax.국가명.map(lambda x:nation_dic[x])
einfomax['원천매체구분코드'] = '1'

einfomax = einfomax.rename(columns = {
    'Date' : '기준일자',
    'Title': '지표명',
    'index': '지표내용',
    'impact':'중요도구분코드'
})

In [264]:
einfomax

,기준일자,Cur,국가명,지표명,지표내용,중요도구분코드,지표값,국가코드,원천매체구분코드
0,20210801,KRW,한국,무역수지,1.76B,2,1.76,410,1
1,20210801,KRW,한국,수입(전년대비),38.2%,2,38.20,410,1
2,20210801,KRW,한국,수출(전년대비),29.6%,2,29.60,410,1
3,20210802,AUD,호주,AIG 제조업지수,60.8,2,60.80,36,1
4,20210802,JPY,일본,제조업 PMI,53.0,2,53.00,392,1
...,...,...,...,...,...,...,...,...,...
11321,20200430,USD,미국,미시간대 1년 물가전망,3.4%,3,3.40,840,1
11322,20200430,USD,미국,미시간대 5년 물가전망,2.70%,3,2.70,840,1
11323,20200430,USD,미국,미시간대 소비자심리지수,88.3,1,88.30,840,1
11324,20200430,USD,미국,미시간대 소비자심리지수(전망),82.7,1,82.70,840,1


#### Naver Finance
    1.Web : https://finance.naver.com/marketindex/
    
    2.수집데이터
        a.환전고시환율 [symbol : exchangeDailyQuote]
            - 원달러환율[symbol : FX_USDKRW]
            - 원엔환율[symbol : FX_JPYKRW]
            - 원유로환율[symbol : FX_EURKRW]
            - 원위안환율[symbol : FX_CNYKRW]
            
        b.세계 기초 자산 시장 [symbol : worldDailyQuote]
            - 국제 금[symbol : CMDT_GC]
            - 서부텍사스유(WTI)[symbol : OIL_CL]
            - 두바이유[symbol : OIL_DU]
            
        c.유가 [symbol : oilDailyQuote]
            - 휘발유[symbol : OIL_GSL]
            
        ㅇ.국내시장금리 [symbol : worldDailyQuote]
            - CD금리(91일)[symbol : IRR_CD91]
            - 콜금리[symbol : IRR_CALL]
            - 국고채(3년)[symbol : IRR_GOVT03Y]
    
    3.url특징
        a. https://finance.naver.com/marketindex/ + 대분류 symbol + .nhn?marketindexCd= + 중분류 symbol
            - 세계 기초 자산 시장 : ?marketindexCd={symbol}&fdtc=2&page={pagenum}
          

In [134]:
dic = {
    'exchangeDailyQuote' : ['FX_USDKRW','FX_JPYKRW','FX_EURKRW','FX_CNYKRW'],
    'worldDailyQuote' : ['CMDT_GC','OIL_CL','OIL_DU'],
    'oilDailyQuote' : ['OIL_GSL'],
    'interestDailyQuote' : ['IRR_CD91','IRR_CALL','IRR_GOVT03Y'],
}

In [135]:
url = []
for k,vs in dic.items():
    for v in vs:
        u = 'https://finance.naver.com/marketindex/' + k + '.nhn?marketindexCd=' + v
        if v in ['CMDT_GC','OIL_CL','OIL_DU']:
            u += '&fdtc=2'
        u += '&page={}'
        url.append(u)
print(url)

['https://finance.naver.com/marketindex/exchangeDailyQuote.nhn?marketindexCd=FX_USDKRW&page={}', 'https://finance.naver.com/marketindex/exchangeDailyQuote.nhn?marketindexCd=FX_JPYKRW&page={}', 'https://finance.naver.com/marketindex/exchangeDailyQuote.nhn?marketindexCd=FX_EURKRW&page={}', 'https://finance.naver.com/marketindex/exchangeDailyQuote.nhn?marketindexCd=FX_CNYKRW&page={}', 'https://finance.naver.com/marketindex/worldDailyQuote.nhn?marketindexCd=CMDT_GC&fdtc=2&page={}', 'https://finance.naver.com/marketindex/worldDailyQuote.nhn?marketindexCd=OIL_CL&fdtc=2&page={}', 'https://finance.naver.com/marketindex/worldDailyQuote.nhn?marketindexCd=OIL_DU&fdtc=2&page={}', 'https://finance.naver.com/marketindex/oilDailyQuote.nhn?marketindexCd=OIL_GSL&page={}', 'https://finance.naver.com/marketindex/interestDailyQuote.nhn?marketindexCd=IRR_CD91&page={}', 'https://finance.naver.com/marketindex/interestDailyQuote.nhn?marketindexCd=IRR_CALL&page={}', 'https://finance.naver.com/marketindex/inter

In [148]:
dic = {'FX_USDKRW' : '원달러환율',
       'FX_JPYKRW':'원엔환율',
       'FX_EURKRW':'원유로환율',
       'FX_CNYKRW':'원위안환율',
       'CMDT_GC':'국제 금',
       'OIL_CL' : '서부텍사스유',
       'OIL_GSL': '휘발유',
       'OIL_DU' : '두바이유',
       'IRR_CD91' : 'CD금리(91일)',
       'IRR_CALL': '콜금리',
       'IRR_GOVT03Y' : '국고채(3년)'}

In [207]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from time import sleep
import json
import urllib.request as req
import os
import re
import numpy as np

In [158]:
def ftn(url):
    symbol = re.findall(r'nhn\?marketindexCd=([A-Z|_|0-9]+)',url)[0]

    page_num = 1
    
    date,index = [],[]
    while 1:
        tmp_url = url.format(page_num)
#         print(tmp_url)
        res = req.urlopen(tmp_url).read().decode("CP949")
        soup = BeautifulSoup(res, "html.parser")
        tmp = soup.find("tbody").find_all('tr')
        if len(tmp) == 0:
            break
        else:
            for tmp in soup.find("tbody").find_all("tr"):
                date.append(tmp.find("td",class_ = "date").text.strip())
                index.append(tmp.find("td",class_ = "num").text.strip())
#         break
    
    df = pd.DataFrame({
        'Date' : date,
        'Index' : index
    })
    df['Cate'] = dic[symbol]
    return df

In [159]:
naverfinance = pd.DataFrame({
    'Date' : [],
    'Index' : [],
    'Cate' : []
})

In [160]:
for u in url:
    naverfinance = pd.concat([naverfinance,ftn(u)])

In [190]:
naverfinance['Date'] = naverfinance.Date.map(lambda x:x.replace(".",""))
naverfinance['원천매체구분코드'] = '2'
naverfinance['국가코드'] = nation_dic['한국']
naverfinance['지표값'] = naverfinance['Index']
naverfinance['지표내용'] = naverfinance['Index']
naverfinance['중요도구분코드'] = None

naverfinance = naverfinance.rename(columns = {
    'Date' : '기준일자',
    'Cate' : '지표명'
})

In [191]:
naverfinance

,기준일자,Index,지표명,원천매체구분코드,국가코드,지표내용,지표값,중요도구분코드
0,20210819,"1,174.10",원달러환율,2,410,"1,174.10","1,174.10",None
1,20210818,"1,170.50",원달러환율,2,410,"1,170.50","1,170.50",None
2,20210817,"1,174.50",원달러환율,2,410,"1,174.50","1,174.50",None
3,20210813,"1,169.00",원달러환율,2,410,"1,169.00","1,169.00",None
4,20210812,"1,163.50",원달러환율,2,410,"1,163.50","1,163.50",None
...,...,...,...,...,...,...,...,...
2,20210813,1.41,국고채(3년),2,410,1.41,1.41,None
3,20210812,1.41,국고채(3년),2,410,1.41,1.41,None
4,20210811,1.41,국고채(3년),2,410,1.41,1.41,None
5,20210810,1.43,국고채(3년),2,410,1.43,1.43,None


#### Naver 거시경제지표
    1.Web : https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=1&ie=utf8&query=%EA%B1%B0%EC%8B%9C%EA%B2%BD%EC%A0%9C%EC%A7%80%ED%91%9C
    
    2.특징
        a. einfomax 와 비슷하나 einfomax에 비해 나라 수가 더 많음
            - 세이셸, 코소보 제외[ZAA009M00에 정의되지 않음]
        b. 정확한 시점을 고를 수 없고 특정 시점에서 이전에 발표한 지표를 크롤링
            - 중복가능 >> dropduplicates


In [162]:
from bs4 import BeautifulSoup
import requests as req
import datetime 
import json
import pandas as pd

In [169]:
delta = datetime.timedelta(days = 1)
yymmdd = datetime.date.fromisoformat("2016-01-05")
Date,Nation,Title,Info1,Info2 = [],[],[],[],[]
while yymmdd <= datetime.date.fromisoformat("2021-07-31"):
    url = "https://m.search.naver.com/p/csearch/content/nqapirender.nhn?where=m&pkid=645&key=dateList&u2=100&u3={}&u4=prev&u1=0".format("".join(yymmdd.isoformat().split("-")))
    soup = req.get(url)
    js = json.loads(soup.text)
    soup = BeautifulSoup(js['html'],"html.parser")

    for tmp in soup.find_all("li",class_ = "anno_type"):
        Date.append(str(yymmdd.year)+"."+tmp.find("strong",class_ = "cm_date").text.replace("\n","").strip().split()[0]) # Date
        Nation.append(tmp.find("i").get("class")[1]) # Nation
        Title.append(tmp.find("div",class_ = "info_title").text) # Info Title
        info = tmp.find("dl",class_ = "info_group").find_all("dd")
        Info1.append(info[0].text)
        if len(info) > 1:
            Info2.append(info[1].text)
        else:
            Info2.append("-")
    yymmdd += delta
    print(yymmdd)
#     break

2016-01-06


In [170]:
naverindex = pd.DataFrame({
    "기준일자":Date,
    "Nation":Nation,
    "Title":Title,
    "Info1":Info1,
    "Info2":Info2})

In [212]:
naverindex = naverindex.drop_duplicates(ignore_index = True)

naverindex['기준일자'] = naverindex.기준일자.map(lambda x:x.replace('.',''))
naverindex = naverindex[naverindex.apply(lambda x: x['Title'].split(" ")[0] in nation_dic, axis = 1)].reset_index(drop = True)
naverindex['국가코드'] = naverindex.Title.map(lambda x:nation_dic[x.split(" ")[0]])
naverindex['중요도구분코드'] = None
naverindex['원천매체구분코드'] = 2
naverindex['지표값'] = naverindex.Info1.map(lambda x:ftn(x))
naverindex = naverindex.rename(columns = {
    'Title' : '지표명',
    'Info1' : '지표내용'
})

In [205]:
def ftn(x):
    if x != x:
        return None#,x
    try :
        return np.float64(x)
    except:
        for num in range(len(x)):
            if x[num].isdigit() or x[num] == "-" or x[num] == "." or x[num] == ",":
                continue
            else:
                break
        return np.float64(x[:num].replace(",",""))#,x[num:]

In [266]:
naverindex

,기준일자,Nation,지표명,지표내용,Info2,국가코드,중요도구분코드,원천매체구분코드,지표값
0,20160102,PER,페루 소비자물가지수(CPI) 전월대비,0.45%,0.10% 상승\n,604,None,2,0.45
1,20160101,KOR,한국 무역수지,7.16B,3.24B 하락\n,410,None,2,7.16
2,20160101,KOR,한국 수입액 전년대비,-19.20%,1.70% 하락\n,410,None,2,-19.20
3,20160101,KOR,한국 수출액 전년대비,-13.80%,9.10% 하락\n,410,None,2,-13.80
4,20160101,CHN,중국 제조업 구매관리자지수(PMI),49.70DI,0.10DI 상승\n,156,None,2,49.70
5,20160101,CHN,중국 비제조업 구매관리자지수(PMI),54.40DI,0.80DI 상승\n,156,None,2,54.40
6,20160101,IND,인도 외환보유고 (미국달러),352.05B,0.94B 상승\n,356,None,2,352.05


#### 데이터 합치기

In [282]:
df = pd.concat([einfomax.loc[:,['기준일자','원천매체구분코드','지표명','지표값','지표내용','중요도구분코드','국가코드']],
           naverfinance.loc[:,['기준일자','원천매체구분코드','지표명','지표값','지표내용','중요도구분코드','국가코드']],
           naverindex.loc[:,['기준일자','원천매체구분코드','지표명','지표값','지표내용','중요도구분코드','국가코드']]])

df.reset_index(drop = True).to_csv("RAA_시장지표.csv",
                                   index = False,encoding="utf-8-sig")

,기준일자,원천매체구분코드,지표명,지표값,지표내용,중요도구분코드,국가코드
0,20210801,1,무역수지,1.76,1.76B,2,410
1,20210801,1,수입(전년대비),38.2,38.2%,2,410
2,20210801,1,수출(전년대비),29.6,29.6%,2,410
3,20210802,1,AIG 제조업지수,60.8,60.8,2,36
4,20210802,1,제조업 PMI,53.0,53.0,2,392
...,...,...,...,...,...,...,...
11417,20160101,2,한국 수입액 전년대비,-19.2,-19.20%,None,410
11418,20160101,2,한국 수출액 전년대비,-13.8,-13.80%,None,410
11419,20160101,2,중국 제조업 구매관리자지수(PMI),49.7,49.70DI,None,156
11420,20160101,2,중국 비제조업 구매관리자지수(PMI),54.4,54.40DI,None,156
